# Backpropagation through Back substitution with a Backslash
Notebook per il seminario d'esame per il corso di Metodi di Approssimazione 

## Algebra degli operatori

Definiamo il tipo di dato custom che rappresenta la funzione lineare

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [2]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator

adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪), X::Matrix) = -X # 𝒪 - X should be -X
*(A::Operator, X::Matrix) = A.op(X)
\(ℐ::typeof(ℐ()), A::Matrix) = A
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

E alcune regole create da me, ma che non attivo ancora

In [ ]:
# Operazioni aritmetiche tra operatori, fatte da me:

# l'identità moltiplicativa di Operator è... l'identità
#Base.one(::Type{Operator}) = ℐ()

# Il prodotto tra operatori è la loro composizione
#*(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)

# Somma tra operatori, definita nel modo ovvio
#+(A::Operator, B::Operator) = Operator(X->(A.op(X) + B.op(X)), X->(A.adj(X) + B.adj(X)), A.sym*" + "*B.sym)

# Differenza tra operatori, definita nel modo ovvio
#-(A::Operator, B::Operator) = A + (-B)

# A quanto pare serve Matrix{Operator} * Operator




In [ ]:
#import Base: convert, copy

# Un Operator è già un Operator
#convert(::Type{Operator}, op::Operator) = op

# In `inv` ci serve un copy constructor a quanto pare
#copy(op::Operator) = op

## Esempi

Vediamo qualche esempio.

In [4]:
# Basic Test
B = [ 1 2; 3 4]
M = [10 1;1 10]
C = [ 2 5; 4 6]

LM = ℒ(M)

ℒ(2, 2)

### `Operator * Matrix`

A parte la negazione `-`, la prima regola che abbiamo scritto è
```
*(A::Operator, X::Matrix) = A.op(X)
```
Ossia, il prodotto tra un operatore e una matrice consiste nell'applicare l'operatore alla matrice. 

Iniziamo guardando la moltiplicazione a sinistra

In [5]:
typeof(LM), typeof(I(2))

(Operator, Diagonal{Bool, Vector{Bool}})

In [6]:
LM * [1 0; 0 1]

2×2 Matrix{Int64}:
 10   1
  1  10

Osserviamo che abbiamo definito il prodotto *a destra* per una matrice. Se moltiplichiamo a sinistra per una matrice, le cose non tornano 

In [7]:
try
    [1 0; 0 1] * LM
catch e
    print("Non si può fare questa moltiplicazione. Ecco l'errore che dà:\n\n")
    showerror(stdout, e)
end

Non si può fare questa moltiplicazione. Ecco l'errore che dà:

MethodError: no method matching *(::Matrix{Int64}, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::AbstractMatrix, ::LinearAlgebra.AbstractQ)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/abstractq.jl:201
  *(::AbstractMatrix, ::LinearAlgebra.AbstractRotation)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/givens.jl:20
  ...


... e ci sono altri piccoli problemini per il fatto che abbiamo usato `Matrix` 

In [8]:
try
    LM * I(2)
catch e
    print("No, non si può fare:\n\n")
    showerror(stdout, e)
end

No, non si può fare:

MethodError: no method matching *(::Operator, ::Diagonal{Bool, Vector{Bool}})
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::Operator, ::Matrix)
   @ Main ~/Scrivania/MA Lab/MA-project/Codice/test/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:10
  *(::Transpose{T, <:AbstractVector} where T, ::Diagonal, ::AbstractVector)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/diagonal.jl:1034
  ...


Non si può fare perché `I` non è una `Matrix` bensì una `AbstractMatrix`:

In [9]:
typeof(I(2)) <: Matrix, typeof(I(2)) <: AbstractMatrix

(false, true)

Ma torniamo alle cose che funzionano

In [10]:
LM * B

2×2 Matrix{Int64}:
 13  24
 31  42

E in effetti è la stessa cosa che fare...

In [11]:
LM.op(B)

2×2 Matrix{Int64}:
 13  24
 31  42

Vediamo ora la moltiplicazione a destra

In [12]:
ℛ(M) * B # i.e. ℛ(M).op(B) 

2×2 Matrix{Int64}:
 12  21
 34  43

In [13]:
B * M # right multiply by M

2×2 Matrix{Int64}:
 12  21
 34  43

La notazione appare controintuitiva, perché lo stesso simbolo `*` viene usato per denotare la moltiplicazione di matrici, e in effetti alla fin fine di un prodotto tra matrici si tratta. Eppure ha senso in realtà: `ℛ(M) * B ` denota l'applicazione dell'operatore `ℛ(M)` a `B`, ovvero `ℛ(M).op(B)`.

Per completezza, vediamo il prodotto di Hadamard

In [14]:
[ℋ(M) * B M .* B]

2×4 Matrix{Int64}:
 10   2  10   2
  3  40   3  40

Ricordiamo che in Julia `.*` vuol dire moltiplicazione *pointwise*

### Trasposizione di operatori

Tra le varie cose, abbiamo scritto
```
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
```
Ovvero abbiamo definito la trasposizione tra operatori. Infatti ricordiamo che in Julia `A'` essenzialmente viene tradotto in `adjoint(A)`.

Ricordiamo inoltre che `adjoint` 
- Fa l'aggiunta "**lazy**". Non "materializza" alcun nuovo oggetto, bensì restituisce una "view" differente sull'oggetto a cui viene applicato
    - Se `B = A'`, modificando `B` si modifica `A`
- E' **involutorio**. Questo "wrapper" viene rimosso quando il metodo viene chiamato due volte, restituendo il "genitore", i.e. l'elemento che è stato trasposto
    - `A'` registra chi è il genitore (`A`) e il tipo di elementi che restituisce dopo la trasposizione (i.e. aggiunto di `eltype(A)`)
- E' **ricorsivo**. `A'` traspone `A` e applica il metodo `adjoint` agli elementi di `A`
    - Concettualmente, `adjoint(A)[i,j] = adjoint(A[j,i])`.



Verifichiamo che la trasposizione di operatori faccia il suo dovere, ossia che 
$$\langle B, \mathcal{L}\,C\rangle=\langle\mathcal{L}'\,B, C\rangle$$
dove $\mathcal{L}'$ denota l'operatore aggiunto $\mathcal{L}^{\top}$

In [15]:
print("<B,ℒC> = $(tr( B'*(ℒ(M)*C) )),\t <ℒ'B,C> = $(tr( (ℒ(M)'*B)' * C ))")   # <ℒ'B,C>

<B,ℒC> = 522,	 <ℒ'B,C> = 522

In termini un po' più "espliciti"

In [16]:
tr(B' * ℒ(M).op(C)), tr( ℒ(M).adj(B)' * C)

(522, 522)

### `Operator ∘ Operator`

Un'altra importante regola che abbiamo scritto è
```
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
```
Che definisce la composizione tra operatori

La composizione fa da prodotto nell'algebra degli endomorfismi di uno spazio vettoriale. 

Quindi potrebbe essere ragionevole denotare la composizione con `*`.

***OSS***: purtroppo non lavoriamo con *endomorfismi* di $\mathbb{R}^{m\times n}$, bensì con omomorfismi tra spazi di matrici. Quindi per essere "sound" dovremmo mettere un controllo sull'ammissibilità dell'operazione. Invece così com'è ora è possibile definire composizioni di omomorfismi che non si possono comporre, e scoprire l'errore solo a runtime.

Comunque, i pezzi di codice sono puramente strumentali a far funzionare il metodo che viene presentato nell'articolo. Questo è soltanto un primo esempio di questo fenomeno. Nella fattispecie, la composizione viene usata solo per costruire gli elementi delle matrici `M` e `ImL`. Quindi, sì, potremmo anche sovraccaricare il prodotto tra omomorfismi...

In [17]:
try
    comp = ℒ(M) * ℒ(M)
catch e
    print("Errore!\n\n")
    showerror(stdout, e)
    println()
end

Errore!

MethodError: no method matching *(::Operator, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::Operator, ::Matrix)
   @ Main ~/Scrivania/MA Lab/MA-project/Codice/test/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:10
  *(::BitMatrix, ::UniformScaling)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/uniformscaling.jl:264
  ...



In [18]:
comp1 = ℒ(M) ∘ ℒ(M)

ℒ(2, 2)∘ℒ(2, 2)

In [19]:
comp1 * [1 0; 0 1]

2×2 Matrix{Int64}:
 101   20
  20  101

fa ciò che ci si aspetta:

In [20]:
(M * M) * [1 0; 0 1]

2×2 Matrix{Int64}:
 101   20
  20  101

Vediamo un altro esempio

In [21]:
comp2 = ℒ(M) ∘ ℛ(C)

ℒ(2, 2)∘ℛ(2, 2)

In [22]:
comp2 * B       # (ℒ(M) ∘ ℛ(C))[B]

2×2 Matrix{Int64}:
 122  209
 230  407

Sarebbe

In [23]:
ℒ(M).op(ℛ(C).op(B))

2×2 Matrix{Int64}:
 122  209
 230  407

ovvero

In [24]:
M * B * C

2×2 Matrix{Int64}:
 122  209
 230  407

**Attenzione!** Potremmo star puntando la pistola contro il nostro piede, e il sistema ci avviserebbe che premendo il grilletto il proiettile parte contro il nostro corpo solo dopo che il mignolino è saltato per aria!

In termini meno poetici, potremmo star componendo omomorfismi che non si possono comporre, e il sistema ce lo lascerebbe fare

In [25]:
# Julia ci lascia inserire il proiettile...
compWrong = ℒ([1 2 3]) ∘ ℒ([2 -1; 1 2])

ℒ(1, 3)∘ℒ(2, 2)

In [26]:
# ... e ci avverte solo dopo lo sparo!
try 
    compWrong * rand(2,3)
catch e
    print("Sparandoti contro il piede, ti farai male al piede!\n\n")
    print("Come dici? Potevo dirtelo prima? Avrei dovuto non lasciarti puntare il fucile?\n")
    print("Beh, e che ne sapevo che avevi intenzione di sparare?!?")
end

Sparandoti contro il piede, ti farai male al piede!

Come dici? Potevo dirtelo prima? Avrei dovuto non lasciarti puntare il fucile?
Beh, e che ne sapevo che avevi intenzione di sparare?!?

## Una semplice rete neurale

In [27]:
using OffsetArrays

# funzione di attivazione
h(x)  = tanh(x)
h′(x) = 1 - h(x)^2


# questa funzione, di fatto, effettua il forward pass
function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

Adesso possiamo costruire la rete

In [28]:
## Un po' di parametri che descrivono la rete...

n = [5,4,3,1]   # contiene [n₀...n_N], le dimensioni dei layer
k = 10          # batchsize
N = length(n)-1 # numero di layer (nascosti + output). Dev'essere positivo

init(sizes...) = 0.01randn(sizes...)    # utility function per inizializzare

init (generic function with 1 method)

In [29]:
## Un po' di parametri degni di questo nome

# Creiamo le matrici dei pesi e i vettori dei bias
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here

X₀ = init(n[1],k)         # batch di k pattern (aka i dati)
y  = init(n[end],k);      # y is what we will compare X_N against (aka l'etichetta)

𝓁(x,y)  = sum(abs2,x-y)/2   # loss (errore quadratico)
𝓁′(x,y) = x .- y;           # derivata della loss (w.r.t. output layer)

X, δ = neural_net(Ws_and_bs,X₀) # Inferenza (aka forward pass)
# X e δ hanno i valori per ogni layer (servono per fare il backward pass)

([[-0.024756438815679994 0.010985011719868971 … -0.007393285557836492 -0.01171477778482939; -0.015591087000339553 -0.015498982415571046 … 0.0005271984013095187 -0.004772752729806914; … ; -0.002616813713831695 0.0006952115009760237 … 0.012425890401561706 -0.005520177532158098; -0.0027179468916055357 -0.01828700761907737 … -0.0009994740033675794 -0.011496960514483685], [-0.002031532466770366 -0.001984718666532716 … -0.0017408477461487372 -0.001989906174250804; 0.002971159723879227 0.0027173654568360853 … 0.002860513577982231 0.002647330326002485; -0.0031455663269039343 -0.0030207946086791794 … -0.0030553354727664347 -0.0030732398776439017; -0.015031594537471937 -0.015186204858684074 … -0.015240922973302604 -0.015197470939293514], [-0.005995967616102618 -0.005994036114886113 … -0.005995862547779353 -0.005992667404583754; 0.008146563097286898 0.008141124958209765 … 0.008141514507831227 0.008141068189279012; -0.010761973643091427 -0.010760650865639678 … -0.010765628486599631 -0.010761097303

Adesso possiamo calcolare il gradiente della loss usando la nostra tecnica

In [30]:
## The diagonal matrix
#M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N])
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i]) ∘ ℛ(ones(1,k))] for i=1:N])

#M = Bidiagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N], [ℋ(δ[i]) ∘ ℛ(ones(1,k)) for i=1:(N-1)], :U)
#M = [M [fill(𝒪, N-1); ℋ(δ[N]) ∘ ℛ(ones(1,k))]]


## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## derivata della loss (rispetto all'output layer)
g = [ fill(𝒪,N-1) ; [𝓁′(X[N],y)] ]      # OSS: con 𝒪() non funziona!

## Finalmente, il gradiente della loss, usando il backslash
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[1.51772136084105e-8 1.510104413240678e-8 … -1.5760357947930958e-8 5.448439555025528e-9; -1.760520401574958e-8 -1.7516750528872358e-8 … 1.8281602804191917e-8 -6.320115697255382e-9; -3.12896369853409e-8 -3.1132617490957864e-8 … 3.2491834958268865e-8 -1.1232561313923825e-8; -4.8810657652529426e-8 -4.8565367238806454e-8 … 5.068555341184913e-8 -1.752216742069191e-8]; [-2.3283258834031002e-6; 2.700794283572557e-6; 4.800121321123293e-6; 7.487960062176237e-6;;];;]
 [[5.005607853425098e-7 -7.835252321711422e-7 8.400422930293392e-7 4.204303787410117e-6; -3.896923162511849e-7 6.099833894637275e-7 -6.539825662439184e-7 -3.2730987570445413e-6; -4.882467419530906e-7 7.642501271782098e-7 -8.193768347726459e-7 -4.1008758277940515e-6]; [-0.0002776355459690965; 0.0002161424593247743; 0.0002708055737867652;;];;]
 [[0.00018695843694106298 -0.00025389464798254507 0.0003356045952579028]; [-0.031181978225140537;;];;]

In [31]:
for i=1:N
    print("∇J[$i]\n")
    print("\tTipo: $(typeof(∇J[i]))\t size: $(size(∇J[i]))\n")
    print("\t\t∇J[$i][1] ha dimensione $(size(∇J[i][1])), e n_$(i) x n_$(i-1) = $(n[i+1])x$(n[i]). Tipo $(typeof(∇J[i][1]))\n")
    print("\t\t∇J[$i][2] ha dimensione $(size(∇J[i][2])), e n_$(i) \t   = $(n[i+1]).   Tipo $(typeof(∇J[i][2]))\n")
    print("\n")
end

∇J[1]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[1][1] ha dimensione (4, 5), e n_1 x n_0 = 4x5. Tipo Matrix{Float64}
		∇J[1][2] ha dimensione (4, 1), e n_1 	   = 4.   Tipo Matrix{Float64}

∇J[2]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[2][1] ha dimensione (3, 4), e n_2 x n_1 = 3x4. Tipo Matrix{Float64}
		∇J[2][2] ha dimensione (3, 1), e n_2 	   = 3.   Tipo Matrix{Float64}

∇J[3]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[3][1] ha dimensione (1, 3), e n_3 x n_2 = 1x3. Tipo Matrix{Float64}
		∇J[3][2] ha dimensione (1, 1), e n_3 	   = 1.   Tipo Matrix{Float64}



### Check

Calcoliamo il gradiente usando il metodo delle differenze finite, per controllare il risultato

In [32]:
#∇Jfd is the gradient calculated with finite differences method

# creiamo uno zero della stessa dimensione del gradiente
∇Jfd = Ws_and_bs*0
ϵ = Ws_and_bs*0
𝜀 = .0001
for i=1:length(Ws_and_bs), wb=1:2
    for j=1:length(ϵ[i][wb])    
        ϵ[i][wb][j] = 𝜀
        # Derivata direzionale approssimata con differenze finite
        ∇Jfd[i][wb][j] = (𝓁(neural_net(Ws_and_bs+ϵ,X₀)[1][N],y) .- 𝓁(neural_net(Ws_and_bs-ϵ,X₀)[1][N],y))/2𝜀
        ϵ[i][wb][j] = .0
    end
end

In [33]:
for i=1:N
    print("∇Jfd[$i]\n")
    print("\tTipo: $(typeof(∇Jfd[i]))\t size: $(size(∇Jfd[i]))\n")
    print("\t\t∇Jfd[$i][1] ha dimensione $(size(∇Jfd[i][1])), e n_$(i) x n_$(i-1) = $(n[i+1])x$(n[i]). Tipo $(typeof(∇Jfd[i][1]))\n")
    print("\t\t∇Jfd[$i][2] ha dimensione $(size(∇Jfd[i][2]))  , e n_$(i) \t     = $(n[i+1]).   Tipo $(typeof(∇Jfd[i][2]))\n")
    print("\n")
end

∇Jfd[1]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[1][1] ha dimensione (4, 5), e n_1 x n_0 = 4x5. Tipo Matrix{Float64}
		∇Jfd[1][2] ha dimensione (4,)  , e n_1 	     = 4.   Tipo Vector{Float64}

∇Jfd[2]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[2][1] ha dimensione (3, 4), e n_2 x n_1 = 3x4. Tipo Matrix{Float64}
		∇Jfd[2][2] ha dimensione (3,)  , e n_2 	     = 3.   Tipo Vector{Float64}

∇Jfd[3]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[3][1] ha dimensione (1, 3), e n_3 x n_2 = 1x3. Tipo Matrix{Float64}
		∇Jfd[3][2] ha dimensione (1,)  , e n_3 	     = 1.   Tipo Vector{Float64}



Interessante notare un leggero mismatch di tipi tra `∇J` e `∇Jfd`, ma restano compatibili. 

In [34]:
# flatten(J) = vcat((x->x[:]).(vcat(J...))...)

function flatten(J) 
    # create a 2N x 1 matrix of matrices [W₁; B₁; ... Wₙ]
    stackedWandB = vcat(J...)       # this is a Matrix{Matrix{Float64}}
    # flatten out the matrices inside the stack
    unstackedStack = (x->x[:]).(stackedWandB)   # this is a Matrix{Vector{Float64}}

    # stack all the vectors that are in the `unstackedStack` matrix
    return vcat(unstackedStack...)
end

flatten (generic function with 1 method)

In [35]:
norm(flatten(∇J)-flatten(∇Jfd))

3.99964722666718e-12

## Errori, e problemi

Il codice proposto è ben lontano dall'essere un sistema completo che permette di definire un'algebra degli operatori. Piuttosto, i metodi sovraccaricati sono quelli strettamente necessari a far funzionare questo esempio (ossia a presentare ciò che l'articolo voleva presentare, che non è poca cosa).

In altri termini, non possiamo fare tutti i conti che ci pare.

### Valutazione di `∇J`

Abbiamo visto che $\nabla J = M^{\top}\cdot (I-L^{\top})^{-1} \cdot g$

In matematica vale la proprietà associativa, ma è bene tenere a mente che i due prodotti della formula sono ben diversi!
- $M^{\top}\cdot(I-L)^{-\top}$: prodotto tra due matrici di operatori. 
    - Per definirlo, bisogna definire un prodotto riga per colonna in cui il prodotto tra gli ""scalari"" è il prodotto tra omomorfismi, ossia la composizione. 
    - Questo prodotto è più vicino alla composizione, come operazione
- $(I-L^{\top})^{-1}\cdot g$: prodotto tra una matrice di omomorfismi e un vettore di matrici. 
    - Nemmeno questo prodotto viene esplicitamente definito, ma non ce n'è bisogno, perché abbiamo definito il prodotto tra un operatore e una matrice (sarebbe l'operazione interna, nel prodotto che qui stiamo considerando). 
    - Questo prodotto è più vicino alla valutazione, come operazione

In [36]:
print("typeof(g) = $(typeof(g))\n")
for i in 1:length(g)
    print("\t typeof(g[$i]) = $(typeof(g[i]))\n")
end

typeof(g) = Vector{Any}
	 typeof(g[1]) = typeof(𝒪)
	 typeof(g[2]) = typeof(𝒪)
	 typeof(g[3]) = Matrix{Float64}


***OSS***: Nel termine noto `g` c'è `𝒪` e non `𝒪()`. Sono due cose molto diverse! (`𝒪()` è un operatore). Vedremo con un altro errore...

Al netto degli zeri `𝒪`, `g` è un array che ha matrici come elementi.

Quello che fa l'algoritmo:
- Calcola `x = ImL' \ g` risolvendo il sistema tramite sostituzione all'indietro
    - Fa operazioni "semplici": Operatore * matrice, somma tra matrici, divisione a sx. per l'operatore identità
    - Restituisce un `Vector{Matrix}`
- Calcola `M' * x` 
    - Fa operazioni semplici: Operatore * matrice, somma tra matrici
    - Restituisce un `Vector{Matrix}`

Tutto questo per dire che **non abbiamo la proprietà associativa, nella valutazione di `∇J`, perché i due prodotti sono diversi**

In [37]:
try 
    M' * ImL'
catch e
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching *(::Adjoint{Operator, Matrix{Operator}}, ::Operator)"

### Inversione di `ImL'`

***OSS***: OVVIAMENTE `M' * ImL'` non è la cosa giusta da fare... è per far vedere che non vale la proprietà associativa... Per poter associare mantenendo la correttezza del calcolo, bisognerebbe esplicitamente invertire `ImL'`.

Ma se per qualche strana ragione *volessimo farlo*?

In [38]:
try
    inv(ImL)
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching Operator(::Operator)"

Il problema è che manca l'identità per gli operatori. Beh, definiamola...

In [39]:
Base.one(::Any) = ℐ()

Già che ci siamo...

In [40]:
# Il prodotto tra operatori è la loro composizione
*(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)

# Somma tra operatori, definita nel modo ovvio
+(A::Operator, B::Operator) = Operator(X->(A.op(X) + B.op(X)), X->(A.adj(X) + B.adj(X)), A.sym*" + "*B.sym)

# Differenza tra operatori, definita nel modo ovvio
-(A::Operator, B::Operator) = A + (-B)

- (generic function with 199 methods)

In [44]:
import Base: convert, copy

convert(::Type{Operator}, op::Operator) = op
copy(op::Operator) = op

copy (generic function with 90 methods)

In [46]:
Operator(op::Operator) = op

Operator

In [47]:
try
    inv(ImL)
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching inv(::Operator)"

Il problema è risolto, ma comunque non siamo in grado di invertire.

***OSS***: possiamo tirare tutte le martellate che ci pare. Prima o poi ci scontreremo con il fatto che `inv(Operator)` non è definito

### In generale, moltiplicazione tra matrici di operatori

---

### Moltiplicare matrici di operatori

**AGGIUSTARE**: Dando le martellate giuste, qualcosa si può fare

In [ ]:
Mnuova = Diagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N])

In [ ]:
*(A::Operator, B::Operator) = A ∘ B

In [ ]:
+(A::Operator, B::Operator) = Operator(X->(A.op(X) + B.op(X)), X->(A.adj(X) + B.adj(X)), A.sym*" + "*B.sym)

In [ ]:
Matrix(ImL) * Matrix(Mnuova)

In [ ]:
ImL * M